## Homework: Introduction | Week 01

In this homework, we'll learn more about search and use Elastic Search for practice. 

## Q1. Running Elastic 

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

> dbcbbbd0bc4924cfeb28929dc05d82d662c527b7

👩🏽‍💻 **EXPLANATION**

To run the version above, run in terminal:

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.17.6
```

<img src="./imgs/h1.png" width="50%">

## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

> `index`


In [2]:
# Start ElasticSearch client
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [3]:
# Indexing documents

from tqdm.auto import tqdm # progress bar

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-homework"

es_client.indices.create(index=index_name, body=index_settings)

# indexing documents into Elasticsearch
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do execute a command on a Kubernetes pod?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

> 44.50


Look at the `_score` field.


In [16]:
def elastic_search(
    query: str,
    *,
    filter_clause: dict | list[dict] | None = None,
    size: int = 5,
    index: str = index_name,
) -> list[dict]:
    """
    Run a relevance-ranked search against `index`, optionally adding a filter.

    Parameters
    ----------
    query : str
        The user’s free-text query that goes into the multi-match.
    filter_clause : dict | list[dict] | None, optional
        Any valid ES `filter` clause, e.g. {"term": {"status": "published"}}
        or a list of such clauses.  `None` → no filter (default).
    size : int, optional
        How many top hits to return (default 5).
    index : str, optional
        Index name (defaults to the global `index_name`).

    Returns
    -------
    list[dict]
        Raw hit objects exactly as they appear in response["hits"]["hits"].
    """

    bool_query: dict[str, any] = {
        "must": {
            "multi_match": {
                "query":  query,
                "fields": ["question^4", "text"],   # boost question field
                "type":   "best_fields"
            }
        }
    }
    
    # Add the filter only when the caller supplied one.
    if filter_clause:
        # filter can be a single dict or already a list; normalise to list
        if not isinstance(filter_clause, list):
            filter_clause = [filter_clause]
        bool_query["filter"] = filter_clause
    
    search_query: dict[str, any] = {
        "size": size,
        "query": {"bool": bool_query},
    }
    
    resp = es_client.search(index=index, body=search_query)
    return resp["hits"]["hits"]


In [17]:
query = "How do execute a command on a Kubernetes pod?"

results = elastic_search(query)

print(f"The score of the top results is {results[0]['_score']:.2f}")


The score of the top results is 44.51


## Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?


* How do I copy files from a different folder into docker container’s working directory?


In [26]:
query = "How do copy a file to a Docker container?"

results = elastic_search(
        query,
        filter_clause={"term":
                        {"course": "machine-learning-zoomcamp"}
        },
        size=3
)

print(f"The third question is: \n\n{results[2]['_source']['question']} ")

The third question is: 

How do I copy files from a different folder into docker container’s working directory? 


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)


```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the `How do I execute a command in a running docker container?` question
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

> 1446


In [29]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [ ]:
# results is the list returned from elastic_search()
context_entries = [
    context_template.format(
        question=hit["_source"]["question"],
        text=hit["_source"]["text"]
    )
    for hit in results
]


In [ ]:
# Separate context entries by two linebreaks (\n\n)
context = "\n\n".join(context_entries)

In [32]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [33]:
prompt = prompt_template.format(
    question="How do I execute a command in a running docker container?",
    context=context
)

In [34]:
# measure the length of the prompt

print(f"The length of the prompt is {len(prompt)}")

The length of the prompt is 1462


## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

> 320

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [35]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

# Count the number of tokens in the prompt
num_tokens = len(encoding.encode(prompt))

print(f"The number of tokens in the prompt is {num_tokens}")

The number of tokens in the prompt is 322


## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

>

Note: you can replace OpenAI with Ollama. See module 2.

In [37]:
from openai import OpenAI

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
     model="gpt-4o",
     messages=[{"role": "user", "content": prompt}],

 )

print(response.choices[0].message.content)

To execute a command in a running Docker container, use the `docker exec` command. First, find the container ID by using the `docker ps` command. Then, execute the desired command in the container with the following syntax:

```sh
docker exec -it <container-id> <command>
```

For example, to start a bash session, you would use:

```sh
docker exec -it <container-id> bash
```


## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [38]:
def estimate_cost(n_reqs, tokens_in, tokens_out,
                  price_in=0.005, price_out=0.015):
    cost_in  = (tokens_in  * n_reqs / 1_000) * price_in
    cost_out = (tokens_out * n_reqs / 1_000) * price_out
    return cost_in + cost_out

# Our case
tokens_in = num_tokens
tokens_out = len(encoding.encode(response.choices[0].message.content))

print(f"The estimated cost of the request is ${estimate_cost(1, tokens_in, tokens_out):.4f}")



The estimated cost of the request is $0.0030
